<a href="https://colab.research.google.com/github/badssu/su_lectures/blob/main/01_Data_Prep/01_Data_prep_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

## Практическа задача

Разполагаме с данни за събития на онлайн ритейл бизнес. Данните са в табличен вид, където всяко събитие представлява отделен запис с неговата дата и време до милисекунди в POSIX формат, вид на събитието, идентификатор на потребителя, идентификатор на сесията на потребителя, възраст на потребителя и операционна система на потребителя.


Задачата ни е да групираме потребителите по поведението си и да идентифицираме тези с нестандартно поведение. Данните са в суров вид, следователно за целта на анализа и групирането се налага предварително те да бъдат почистени от грешни записи, както и да се подходи към изграждане на обяснителни характеристики за поведението на посетителите на сайта.

## Прочитане и първоначално почистване

#### Прочитане
Нека започнем с прочитане на данните и зареждането им в pandas таблица като използваме read_csv метода.

In [ ]:
df = pd.read_csv('https://github.com/badssu/su_lectures/raw/main/01_Data_Prep/sessions_and_events_raw_part0.csv', index_col=0)

In [ ]:
df.head()

Тъй като данните са ни в няколко csv файла ще използваме map функцията на python, с която можем да приложим една функция върху масив.

In [ ]:
df = pd.concat(map(lambda x: pd.read_csv(x, index_col=0), [f'https://github.com/badssu/su_lectures/raw/main/01_Data_Prep/sessions_and_events_raw_part{i}.csv' for i in range(8)]))

Разглеждане на основни характеристики:

In [ ]:
(df.describe() # get summary statistics
   .apply(lambda s: s.apply('{0:.0f}'.format))) # apply number formating for a cleaner view

In [ ]:
# how big is our table
len(df)

#### Липсващи данни

Липсващите данни са често явление и правилното справяне с тях е ключово за всички последващи стъпки в моделирането.

Когато открием наличието на липсващи данни трябва да си отговорим на следните въпроси, за да решим как най-добре да се справим с проблема:

1. Напълно случайно ли лиспват данните или тяхната липса е резултат на някакъв процес?
* Напълно случайно липсващи - може спокойно да се подходи или към филтриране или към популиране
* Неслучайно липсващи - липсата на данни ни носи информация за допълнителен процес. В такъв случай е добре допълнително да се закодира информацията, че на даденото място са липсвали данни, преди да се популират със стойности
2. Какъв е обемът на засегнатите записи?
* При много малък обем на засегнати данни, то филтрирането им не би довело до големи загуби.
3. Кои последващи характеристики биха били повлияни от липсващи данни?
* При решението за филтриране или запълване трябва добре да се помисли какви са ни обяснителните характеристики и използвания прогнозен модел. Възможно е при премахване на частични данни да се афектират характеристики разчитащи на бройка и средни стойности.
4. Възможно ли е популирането на липсващите данни, чрез допълнителна логика?
* За да е възможна работата в последващите стъпки с лиспващи данни, те често биват запълвани с:
  * Средна или медианна стойност
  * Най-често срещана стойност
  * Минимална или максимална стойност
  * Стойност извън разпределението пр. -99 (подходящо единствено за една малка част от моделите)
  * Последно срещана стойност (при времеви редове)
  * Предсказана стойност от друг модел

* Преди да се вземе решение за начина на запълване на липсващите данни, трябва да вземем предвид тяхното естество и видът модел, който ще работи с тях. При работа с времеви редове бихме подходили по напълно различен начин, отколкото при работата с регресионна задача. Също така подходът за запълване при работа с линейна регресия би бил различен от този при работа с дръвче.

5. Крайно необходимо ли е да се работи с засегнатите данни?

* Важно е да знаем, че не сме длъжни да използваме всички налични пред нас данни. Мног често разполагаме с повече данни, от колкото ни е необходимо и част от задачата ни е да решим с кои трябва и с кои си заслужава да работим. В случаите, в които не можем да гарантираме качество на дадени данни, то по-добре би било да не фокусираме услията си върху тях (garbage in - garbage out). 


In [ ]:
# how many missings per column
df.isna().agg('sum').head(10)

#### 16 липсващи visitorid
Тъй като фокус на нашето изследване е да групираме потребителите по тяхното поведение, записите с лиспваща информация за visitorid не са подходящи за работа и трябва да ги филтрираме.

In [ ]:
df = df[~df.visitorid.isna()]

#### 38 липсващи itemid

Записите с липсващи стойности за продукт могат да бъдат филтрирани. Тъй като фокус на изследването ще са потребителите то при калкулациите на характеристики за тях, липсващите данни ще доведат то грешно сметнати стойности. Заради това се налага да идентифицираме кои потребители са засегнати и да се премахнат всички събития свързани с тях.

In [ ]:
# get the affected visitorids as a list
affected_visitors = df[df.itemid.isna()].visitorid.unique()
# filter the dataframe to exclude these users
df = df[~df.visitorid.isin(affected_visitors)]

#### 279278 липсващи os

Преди да филтрираме записите с липсваща информация за операционна система и да загубим голяма част от данните следва да се запитаме защо тези данни липсват? 
Причината да за липсваща информация е, това че част от потребителите са използвали функция на браузърите си да не дават такава информация, когато посещават уебсайтове. Тъй като това е съзнателно действие, то също е част от потребителското поведение. В този случаи факта че липсва информация сам по себе си носи информация. За това ще закодираме липсващите стойности като отделна категория със стойност 'N/A'.

In [ ]:
df.os = df.os.fillna('N/A')

#### 2737857 липсващи transactionid

От описанието на данните знаем, че тези стойности са популирани само в случай, че събитието е трансакция. Следователно липсващите стойности не са проблем, стига да знаем как да пресмятаме правилно агрегатни характеристики за потербителите.

#### 560347 липсващи age

Отново ако решим да филтрираме ще загубим голяма част от данните.
Поради това ще се насочим към това да популираме липсващите записи.

В нашият случаи ще подходим с използване на средна стойност. Преди да направим това, обаче, следва да отбележим и че имаме неправилни стойности в полето (отрицателна възраст или твърде голяма възраст). В тези случаи отново можем или да филтрираме или да запълним с други данни. Тъй като са малък брой записи ще премахнем невалидните стойности.

Допълнително обаче, знаем и че причините за липсващи данни не са случайни, а са резултат на потребителско поведение. За това ще изградим и нова бинарна колона, която да описва дали наблюдението е липсвало или не.

In [ ]:
# number of invalid ages
len(df[(df.age <= 0) | (df.age > 120)])

In [ ]:
# filter invalid ages
df = df[~((df.age <= 0) | (df.age > 120))]
# calculate the mean age
mean_age = round(df.age.mean(), 0)
# calculate a boolean column indicating whether the age is missing
df['missing_age'] = df.age.isna().astype('int')
# impute missing ages to be the mean age
df.age = df.age.fillna(mean_age)

In [ ]:
# how many missings per column
df.isna().agg('sum').head(10)

#### Дублирани стойности

Често срещан проблем със сурови данни са дублираните записи.

Преди да идентифицираме дубликати като проблем и да решим как да подходим трябва да си изясним кои колони формират ключа за нашата таблица и съответно да потърсим уникални стойности за тях.

Както и при липсващи и грешни стойности сме изправени пред въпроса дали да филтрираме засегнатите записи, да агрегираме стойностите и осигурим уникалност или да оставим таблицата така.


In [ ]:
# how many identical duplicates are in the dataframe
len(df.drop_duplicates())

In [ ]:
# we drop the identical duplicates as they are unexpected for this table
df = df.drop_duplicates()

In [ ]:
df.count()

In [ ]:
df.nunique()

Можем да забележим, че трансакциите не съдържат напълно уникални стойности, което първоначално изглежда като неочакван резултат.

Нека проверим дали тези записи са дублирани трансакции между потребители или между продукти.


In [ ]:
by_transaction = (df.groupby('transactionid').agg(count_records=('timestamp', 'count'),
                                                  count_unique_visitors=('visitorid', 'nunique'),
                                                  count_unique_items=('itemid', 'nunique'))
                                             .reset_index())
by_transaction = by_transaction[(by_transaction.count_records >= 2) & (~by_transaction.transactionid.isna())]
print(f'Number of transaction ids that have more than one visitor id: {np.sum(by_transaction.count_unique_visitors >= 2)}')
print(f'Number of transaction ids that have more than one item id: {np.sum(by_transaction.count_unique_items >= 2)}')

Изглежда дубликатите са заради факта, че ако клиент закупи повече от един артикул наведнъж, всеки от тях получава запис за трансакционно събитие със същия идентификатор. Това е нормално поведение, следователно няма необходимост от мерки за подсигуряване на уникалност.

#### Работа с обекти за време

В таблицата има налична информация за времето на събитието в POSIX формат. 

Можем да използваме функцията to_datetime на pandas модула, за да я конвертираме в datetime обект. Важно е да отбележим, че стойностите са в милисекунди.

In [ ]:
df.timestamp = pd.to_datetime(df.timestamp, unit='ms')

In [ ]:
df.head()

# Изграждане на обяснителни характеристики


#### Закодиране на категорийни характеристики

За да можем да работим с категорийни харакеристики, както за моделиране, така и за агрегиране, следва да ги закодираме в подходящ за работа формат.

Възможни са различни подходи към категорийните характеристики в зависимост от вида на категориите и вида моделиране.

* Числово закодиране - задаване на целочислени индекси на всяка от категориите

  куче, котка, риба -> 1, 2, 3

  * При работа с параметрични модели и неординални данни такъв тип закодиране не е подходящо, защото предполага че една котка = 2 * куче, което няма смислена стойност
  * При работа с ординални данни е възможен такъв подход, но трябва да се има предвид, че не е оптимален, тъй като най-често дистанцията между категориите не е еднаква, следователно не можем да твърдим, че юноша = 3 * бебе (ординални са категориите бебе, дете, юноша, възрастен)

* Едночислово закодиране - при едночисловото закодиране от К категории се създава вектор с дължина К. Всеки индекс от вектора отговаря за една категория и ако дадения ред е от категория М, то на позиция М имаме 1, а на всички останали позиции имаме 0.

куче, котка, риба -> [1, 0, 0], [0, 1, 0], [0, 0, 1]

  * Важно е при едночисловото закодиране да се премахне една от категориите, защото се получава повтаряне пълна зависимост на едни характеристики от други (мултиколинеарност)
  * Едночисловто закодиране увеличава значително размерността на данните при наличието на множество категории

* Описателно закодиране - при работа с категории с твърде висока кардиналност е възможно да използваме познанието си за бизнеса и да потърсим други обяснителни характеристики (вкл. други категории), които могат да опишат в някаква степен значението и на дадената категория. Пр. вместо да използваме модел на телефон, поради множеството видове и постоянното излизане на нови модели, можем да използваме други характеристики като: марка, размер на дисплей, операционна система и др.

* Научено закодиране - при наученото закодиране обикновено използваме предварително трениран модел, който може от категорията да ни даде уникален вектор. Често за целта се използват невронни мрежи върху текстови или снимкови данни.

Едно от най-популярните закодирания е едночисловото закодиране, в което превръщаме категориите в бинарен вектор отбелязващ, в коя категория е дадения запис.

In [ ]:
df = pd.concat([df, pd.get_dummies(df.event, prefix='event')], axis=1)
df = pd.concat([df, pd.get_dummies(df.os, prefix='os')], axis=1)

In [ ]:
df.head(2)

### Характеристики за сесиите

In [ ]:
# calculating session start, end and number of views
df['has_transaction'] = (~df.transactionid.isna()).astype('int')
by_session = df.groupby('session_id').agg(
    session_start=('timestamp', 'min'),
    session_end=('timestamp', 'max'),
    count_view=('event_view', 'sum'),
    count_addtocart=('event_addtocart', 'sum'),
    count_transaction=('transactionid', 'nunique'),
    count_unique_visited_items=('itemid', 'nunique'),
    has_transaction=('event_transaction', 'max')
).reset_index()
# calculating session duration
by_session['session_duration_minutes'] = (by_session.session_end - by_session.session_start).apply(lambda x: x.seconds / 60)
by_session['avg_view_time_seconds'] = (by_session.session_duration_minutes * 60) / by_session.count_view

In [ ]:
# checking the sessions with most views
# notice the first 3 sessions have a lot of events with 0 duration. These seem to be some bot activity so we need to remove them
by_session.sort_values(['count_view'], ascending=False).head(10)

In [ ]:
# filter the bot activity
by_session = by_session[by_session.count_view <= 500]

In [ ]:
# checking the sessions with most duration
by_session.sort_values(['session_duration_minutes'], ascending=False).head(10)

In [ ]:
# aggregating to the user level
by_session_by_visitor = df[['visitorid', 'session_id']].merge(by_session.drop(['session_start', 'session_end'], axis=1), on='session_id', how='inner')
by_session_by_visitor = (by_session_by_visitor
                           .groupby('visitorid')
                           .agg(
                             avg_views_per_session=('count_view', 'mean'),
                             avg_unique_items_per_session=('count_unique_visited_items', 'mean'),
                             avg_session_duration_minutes=('session_duration_minutes', 'mean'),
                             avg_page_view_time_seconds=('avg_view_time_seconds', 'mean')  
                           )
                           .reset_index())

### Характеристики за покупките


In [ ]:
# building how frequeny a user buys an item
visitor_buying_frequency = \
(df[~df.transactionid.isna()]
  .groupby(['visitorid', 'itemid'])
  .agg(unique_transactions=('transactionid', 'nunique'))
  .reset_index()
  .groupby('visitorid')
  .agg(average_buying_frequency=('unique_transactions', 'mean'))
  .reset_index())

In [ ]:
visitor_buying_frequency.sort_values('average_buying_frequency', ascending=False).head()

In [ ]:
# building average basket size of transactions per customer
visitor_basket_size = \
(df[~df.transactionid.isna()]
  .groupby(['visitorid', 'transactionid'])
  .agg(unique_items=('itemid', 'nunique'))
  .reset_index()
  .groupby('visitorid')
  .agg(average_basket_size=('unique_items', 'mean'))
  .reset_index())

In [ ]:
visitor_basket_size.sort_values('average_basket_size', ascending=False).head()

### Характеристики за време



In [ ]:
min_datetime = df.timestamp.min()
max_datetime = df.timestamp.max()
print(f'min_datetime: {min_datetime}')
print(f'max_datetime: {max_datetime}')

In [ ]:
# calculating visitor age
visitor_age = \
(df.groupby('visitorid')
  .agg(first_session_time=('timestamp', 'min'),
       last_session_time=('timestamp', 'max')))
visitor_age['visitor_first_seen_days'] = visitor_age.first_session_time.apply(lambda x: (max_datetime - x).days)
visitor_age['visitor_inactive_time'] = visitor_age.last_session_time.apply(lambda x: (max_datetime - x).days)
visitor_age['visitor_active_period'] = visitor_age.visitor_first_seen_days - visitor_age.visitor_inactive_time

### Характеристики за събития и трансакции

In [ ]:
by_visitor = df.groupby('visitorid').agg(
    count_events=('event', 'count'),
    count_items=('itemid', 'nunique'),
    count_sessions=('session_id', 'nunique'),
    count_view=('event_view', 'sum'),
    count_addtocart=('event_addtocart', 'sum'),
    count_transaction=('transactionid', 'nunique'),
    sum_transaction=('event_transaction', 'sum'),
    os=('os', 'first'),
    age=('age', 'first')
).reset_index()
by_visitor['add_to_cart_rate'] = by_visitor.count_addtocart / by_visitor.count_view
by_visitor['transaction_rate'] = by_visitor.sum_transaction / by_visitor.count_addtocart

# clip values due to partial sessions
by_visitor.transaction_rate = by_visitor.transaction_rate.clip(upper=1.0)
by_visitor.add_to_cart_rate = by_visitor.add_to_cart_rate.clip(upper=1.0)

by_visitor.transaction_rate = by_visitor.transaction_rate.fillna(value=0.0)
by_visitor.add_to_cart_rate = by_visitor.add_to_cart_rate.fillna(value=1.0)

In [ ]:
by_visitor.sort_values(['count_sessions', 'count_events'], ascending=[False, False]).head(10)

### Събиране на всички характеристики

In [ ]:
by_visitor = by_visitor.merge(visitor_age, on='visitorid', how='inner')
by_visitor = by_visitor.merge(visitor_basket_size, on='visitorid', how='left') # note that since this is only for users with transaction it has to be a left join
by_visitor = by_visitor.merge(visitor_buying_frequency, on='visitorid', how='left') # note that since this is only for users with transaction it has to be a left join
by_visitor = by_visitor.merge(by_session_by_visitor, on='visitorid', how='inner')

In [ ]:
# fill missing features for users without a transaction
by_visitor.average_basket_size = by_visitor.average_basket_size.fillna(0)
by_visitor.average_buying_frequency = by_visitor.average_buying_frequency.fillna(0)

In [ ]:
by_visitor.head(5)

In [ ]:
by_visitor.describe().apply(lambda s: s.apply('{0:.0f}'.format))

## Групиране на потребители

In [ ]:
data = by_visitor[['count_events',
                   'count_items',
                   'count_sessions',
                   'count_view',
                   'count_addtocart',
                   'count_transaction',
                   'sum_transaction',
                   'add_to_cart_rate',
                   'transaction_rate',
                   'visitor_first_seen_days',
                   'visitor_inactive_time',
                   'visitor_active_period',
                   'average_basket_size',
                   'average_buying_frequency',
                   'avg_views_per_session',
                   'avg_unique_items_per_session',
                   'avg_session_duration_minutes',
                   'age'
                   ]]


In [ ]:
pca = PCA(n_components='mle')
dim_reduced = pca.fit_transform(data)

In [ ]:
kmeans = KMeans(n_clusters=6, max_iter=1000).fit(dim_reduced)
data["clusters"] = kmeans.labels_

In [ ]:
# supress warnings
import warnings
warnings.filterwarnings('ignore')
# plot distribution plots of some of the metrics
metrics = ['count_events', 'count_items', 'count_sessions', 'visitor_first_seen_days', 'age']
log_scales = ['count_events', 'count_items', 'count_sessions', 'count_view', 'count_transaction']
for metric in metrics:
  hist_data = [data[data.clusters == i][metric] for i in range(7)]
  #plt.xlim(0, 10_000)
  plt.figure(figsize=(10, 5))
  if metric in log_scales:
    plt.xscale('log')
  [sns.distplot(x, bins=30) for x in hist_data]
  plt.title(metric)
  plt.show()

In [ ]:
data[['count_events',
     'count_sessions',
      'count_transaction',
      'average_basket_size',
      'average_buying_frequency',
      'age',
      'visitor_first_seen_days',
      'clusters']].groupby('clusters').agg(['count', 'sum', 'min', 'max', 'mean', 'median'])